In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# !pip install geopandas

In [22]:
import pandas as pd
import geopandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [9]:
# df2021 = pd.read_csv(folder + 'Turnstile_Usage_Data__2021.csv')
# df2021.to_hdf(path_or_buf= folder + 'Turnstile_Usage_Data__2021.h5', key='df2021', mode='w', complevel=9)
# df = pd.read_hdf(folder + 'Turnstile_Usage_Data__2021.h5', 'df2021')
# df2021.to_pickle(folder + "Turnstile_Usage_Data__2021.pkl")

# Data processing

## Input data

In [8]:
folder = '/content/gdrive/MyDrive/Colab_Notebooks/MTA_turnstile_data/'

In [17]:
df = pd.read_pickle(folder + "Turnstile_Usage_Data__2021.pkl")

Take a quick look of the dataframe


In [23]:
df

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/16/2021,00:00:00,REGULAR,7603117,2599984
1,A002,R051,02-00-01,59 ST,NQR456W,BMT,07/16/2021,00:00:00,REGULAR,6729128,1534591
2,A002,R051,02-03-00,59 ST,NQR456W,BMT,07/16/2021,00:00:00,REGULAR,1441306,5445696
3,A002,R051,02-03-01,59 ST,NQR456W,BMT,07/16/2021,00:00:00,REGULAR,1620576,2531081
4,A002,R051,02-03-02,59 ST,NQR456W,BMT,07/16/2021,00:00:00,REGULAR,46664,64433
...,...,...,...,...,...,...,...,...,...,...,...
5023342,PTH17,R541,01-00-05,THIRTY THIRD ST,1,PTH,01/02/2021,23:55:00,REGULAR,404742,298718
5023343,PTH22,R540,00-00-05,PATH NEW WTC,1,PTH,01/02/2021,23:55:58,REGULAR,48921,36067
5023344,PTH05,R543,00-01-06,EXCHANGE PLACE,1,PTH,01/02/2021,23:55:59,REGULAR,149433,38075
5023345,PTH02,R544,00-00-05,HARRISON,1,PTH,01/02/2021,23:57:50,REGULAR,186801,12741


'Exits' label has extra space, clean it out

In [26]:
df.columns

Index(['C/A', 'Unit', 'SCP', 'Station', 'Line Name', 'Division', 'Date',
       'Time', 'Description', 'Entries',
       'Exits                                                     '],
      dtype='object')

In [27]:
df.columns = df.columns.str.replace(' ', '')

## Add 2 more columns: Turnstile and Datetime
- 'Turnstile' is to identify a <ins>*unique turnstile*</ins>. The data is accumulated, so we need to find a specific turnstile to calculate the entries/exits.
- 'Datetime' is to *combine the date and time* for the convenience of data processing

In [28]:
df['Turnstile'] = df['C/A'] + ' ' + df['Unit'] + ' ' + df['SCP'] + ' ' + df['Station']
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%m/%d/%Y %H:%M:%S')

In [29]:
df.Turnstile.unique().shape

(5050,)

There are 5050 *unique* turnstiles

Remove some duplicated rows, as explained in MTA documents (because of many kinds of reasons)

In [30]:
df.drop_duplicates(subset=['Turnstile', 'Datetime'], inplace=True)

Sort the dataframe with turnstile and datetime

In [31]:
df_sort_datetime = df.groupby(['Turnstile', 'Datetime'],as_index=False).first()

## Calculate increment at each station at each time period

Creat new columns 'Datetime_prev', 'Entries_prev', 'Exits_prev' to store the previous record for a specific turnstile

In [32]:
df_sort_datetime[['Datetime_prev', 'Entries_prev', 'Exits_prev']] = \
    df_sort_datetime.groupby(['Turnstile'])[['Datetime', 'Entries', 'Exits']].apply(lambda row: row.shift(1))

Drop the *NA* data, those are the rows at the beginning of 2021 which don't have a previous record (however, it will have one when inputing the 2020 data)

In [33]:
df_sort_datetime.dropna(inplace=True)

Calculate the entries/exits change in a time period

In [34]:
df_sort_datetime['Entries_change'] = df_sort_datetime['Entries'] - df_sort_datetime['Entries_prev']
df_sort_datetime['Exits_change'] = df_sort_datetime['Exits'] - df_sort_datetime['Exits_prev']

Calculate the time period in the unit of hours and calculate the hourly average entries/exits change

In [35]:
df_sort_datetime['Duration_hr'] = (df_sort_datetime['Datetime'] - df_sort_datetime['Datetime_prev']) / pd.Timedelta('1h')
df_sort_datetime['Entries_change_hourly'] = df_sort_datetime['Entries_change'] / df_sort_datetime['Duration_hr']
df_sort_datetime['Exits_change_hourly'] = df_sort_datetime['Exits_change'] / df_sort_datetime['Duration_hr']

Reorder the column names

In [36]:
df_sort_datetime = df_sort_datetime[['Turnstile', 'Datetime_prev','Datetime', 'Duration_hr',
                                     'Entries_prev', 'Entries', 'Entries_change', 'Entries_change_hourly',
                                     'Exits_prev', 'Exits', 'Exits_change', 'Exits_change_hourly',
                                     'C/A', 'Unit', 'SCP', 'Station', 
                                     'LineName', 'Division', 'Date','Time', 'Description']]

For some reason, there are negative changes in both entries and exits.

For this demo work, I just simply remove those lines.

In [115]:
df_sort_datetime = df_sort_datetime[df_sort_datetime['Entries_change'] >= 0]
df_sort_datetime = df_sort_datetime[df_sort_datetime['Exits_change'] >= 0]

Add up all the turnstile data (at each time) in the same station

In [43]:
df_station = df_sort_datetime.groupby(['Station', 'Datetime'], 
                                      as_index=False)[['Entries_change', 
                                                       'Exits_change', 
                                                       'Entries_change_hourly', 
                                                       'Exits_change_hourly']].agg('sum')

Since not all the turnstile upload data at the same time, but the majority of them update every 4 hours. So, cut the data into 6 periods for the convenience of data processing

In [44]:
df_station['Timegrp'] = pd.cut(
    pd.to_datetime(df_station.Datetime), 
    pd.period_range(start='2021-01-02 00:00:00', end='2021-07-17 00:00:00', freq='4H').to_timestamp())

Combine the data during the same time period at the same station

In [45]:
df_station = df_station.groupby(['Station', 'Timegrp'])[['Entries_change', 'Exits_change', 
                                                         'Entries_change_hourly', 
                                                         'Exits_change_hourly']].agg('sum')

In [ ]:
df_station

# Bubble map: example of entry data during (2021-01-03 00:00:00, 2021-01-03 04:00:00]

Now, focus on the entry data

In [185]:
df_station_entries = df_station[['Entries_change_hourly']]

Unstack for plotting purpose

In [186]:
df_station_entries_plt = df_station_entries.unstack(level = 0)

Load station location data. Only need the GFTS data.
Then, input the hourly average entries change, link with the GFTS data

(note that this method is not very accurate, since the station names are not exactly the same in the two dataframes, even they both come from MTA)

(in the latter work, the station name match needs to be fine-tuned)

Creat a bubble map showing how many commuters are entering subway stations at a certain time period.

Here is the example of:

In [187]:
period_index = 6
df_station_entries_plt.index[period_index]

Interval('2021-01-03', '2021-01-03 04:00:00', closed='right')

In [190]:
df_station_location = pd.read_csv(folder + 'station_location.csv')
df_station_location.columns = df_station_location.columns.str.replace(' ', '_')
dsl_geo = df_station_location[['Stop_Name', 'GTFS_Latitude', 'GTFS_Longitude']].copy()
dsl_geo.sort_values(by='Stop_Name', inplace=True)
dsl_geo.drop_duplicates('Stop_Name', inplace=True)
dsl_geo['Average_Entries'] = df_station_entries_plt.iloc[period_index].values.copy()

bm_enter = folium.Map(location=[40.73,-73.95], tiles="OpenStreetMap", zoom_start=11.25)
for i in range(0,dsl_geo.shape[0]):
   folium.CircleMarker(
      location=[dsl_geo.iloc[i]['GTFS_Latitude'], dsl_geo.iloc[i]['GTFS_Longitude']],
      popup=dsl_geo.iloc[i]['Stop_Name'],
      radius=float(dsl_geo.iloc[i]['Average_Entries'])/10,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(bm_enter)

bm_enter

Let's see how many people were exiting at which stations at the same time period

In [193]:
df_station_exits_plt = df_station[['Exits_change_hourly']].unstack(level = 0)
df_station_entries_plt.index[period_index]

dsl_geo['Average_Exits'] = df_station_exits_plt.iloc[period_index].values.copy()

bm_exit = folium.Map(location=[40.73,-73.95], tiles="OpenStreetMap", zoom_start=11.25)
for i in range(0,dsl_geo.shape[0]):
   folium.CircleMarker(
      location=[dsl_geo.iloc[i]['GTFS_Latitude'], dsl_geo.iloc[i]['GTFS_Longitude']],
      popup=dsl_geo.iloc[i]['Stop_Name'],
      radius=float(dsl_geo.iloc[i]['Average_Exits'])/10,
      color='blue',
      fill=True,
      fill_color='blue'
   ).add_to(bm_exit)

bm_exit

now we see how many commuters are entering subway stations city-wide between (2021-01-02 00:00:00, 2021-01-02 04:00:00]

# Deep Piano
## Train classical piano music midi data with a LSTM sequence model
- midi source: https://magenta.tensorflow.org/datasets/maestro

<ins>***Here are some samples after learning Mozart's piano musics.***</ins>

(You may need to log in to play)



In [194]:
import IPython

In [195]:
pianofolder = '/content/gdrive/MyDrive/Colab_Notebooks/deeppiano/result/'

In [197]:
IPython.display.Audio(pianofolder+ 'result_fs(10)-seq_len(100)-batch_music(16)-random_pick(True)-na(1024)-epochs(200).wav')

Output hidden; open in https://colab.research.google.com to view.

In [198]:
IPython.display.Audio(pianofolder + 'result_fs(10)-seq_len(100)-batch_music(16)-random_pick(True)-na(512)-epochs(200).wav')

Output hidden; open in https://colab.research.google.com to view.

In [199]:
IPython.display.Audio(pianofolder + 'result_fs(10)-seq_len(100)-batch_music(32)-random_pick(True)-na(1024)-epochs(200).wav')

Output hidden; open in https://colab.research.google.com to view.

In [200]:
IPython.display.Audio(pianofolder + 'result_fs(10)-seq_len(100)-batch_music(16)-start_ind(16)-random_pick(True)-na(256)-epochs(200).wav')

Output hidden; open in https://colab.research.google.com to view.